In [1]:
!pip install ultralytics

^C



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\jeeva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/914.7 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/914.7 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 914.7/914.7 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
    --------------------------------------- 2.6/204.2 MB 18.9 MB/s eta 0:00:11
   - -------------------------------------- 5.8/204.2 MB 14.7 MB/s eta 0:00:14
   - -------------------------------------- 9.4/204.2 MB 15.8 MB/s eta 0:00:13
   -- ------------------------------------- 12.8/204.2 MB 16.1 MB/s eta 0:00:12
   --- ------------------------------------ 16.3/204.2 MB 16.2 MB/s eta 0:00:12
   --- ------------------------------------ 19.7/204.2 MB 16.3 MB/s eta 0:00:12
   ---- ----------------------------------- 21.8/204.2 MB 15.5 MB/s eta 0:0

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

results = model.train(data="coco8.yaml", epochs=100, imgsz=640)